# PyTorch Deep Dive: Advanced Topics & Best Practices

You can build models. Now let's build them **professionally**.

In this notebook, we will learn the tools that make PyTorch scalable and robust.

## Learning Objectives
- **The Vocabulary**: What is a "Dataset", "DataLoader", "Batch Size", and "Checkpoint"?
- **The Intuition**: Why we eat with a spoon (Batches) instead of a shovel.
- **The Practice**: Using `torch.utils.data` to handle massive datasets.
- **The Safety**: Saving and Loading your model so you don't lose work.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

torch.manual_seed(42)

## Part 1: The Vocabulary (Definitions First)

### 1. Dataset (`torch.utils.data.Dataset`)
- A class that knows where your data is and how to get one item.
- It must implement `__len__` (How big am I?) and `__getitem__` (Give me item #5).
- Analogy: A Librarian who knows where every book is.

**The `__getitem__` contract**: This method receives an integer index and must return a single sample. PyTorch never calls `__getitem__` with a list of indices — batching is handled by the DataLoader. Your job is to make this method as fast as possible because it's called thousands of times during training.

### 2. DataLoader (`torch.utils.data.DataLoader`)
- A worker that grabs items from the Dataset and bundles them into Batches.
- It handles shuffling and parallel loading (using multiple CPU cores).
- Analogy: A Delivery Driver who packs books into boxes and brings them to you.

**Key parameters you should understand**:
- `num_workers`: How many CPU processes load data in parallel. Set to 0 for debugging, 2-8 for training. Too many workers waste memory.
- `pin_memory`: Set to `True` when using a GPU. Pins data in CPU memory for faster GPU transfer.
- `collate_fn`: Custom function to combine samples into a batch. Needed when samples have different sizes (e.g., variable-length text).
- `drop_last`: Whether to drop the last incomplete batch. Important for BatchNorm layers that need consistent batch sizes.

### 3. Batch Size
- How many items the DataLoader puts in one box.
- **Small Batch (e.g., 1)**: Noisy gradient updates, slow training, but better generalization (acts as regularization).
- **Large Batch (e.g., 1000)**: Stable gradient updates, memory hungry, but can converge to sharp minima (worse generalization).
- **Sweet Spot**: Usually 32, 64, or 128 for most tasks.

**Why batch size affects generalization**: Small batches add noise to gradient estimates, which helps the optimizer escape sharp local minima and find flatter minima that generalize better. This is why simply scaling to huge batches (without tricks like learning rate warmup) often hurts test accuracy.

### 4. Checkpoint
- A file containing the model's weights at a specific point in time.
- Analogy: A "Save Game" file.

**What to save in a checkpoint** (for proper resumption):
```python
checkpoint = {
    'model_state_dict': model.state_dict(),       # Weights
    'optimizer_state_dict': optimizer.state_dict(), # Optimizer momentum
    'epoch': epoch,                                 # Where you stopped
    'loss': loss,                                   # Current loss
    'lr_scheduler': scheduler.state_dict(),         # LR schedule state
}
```
Saving only `model.state_dict()` is fine for inference, but for resuming training you need the optimizer state too — otherwise Adam's momentum and variance accumulators reset to zero.

## Part 2: The Intuition (Spoon vs Shovel)

Why do we use Batches?

Imagine you need to eat a mountain of rice (The Dataset).
- **Batch Size = 1 (SGD)**: Eating one grain at a time. You will starve before you finish.
- **Batch Size = All (Batch GD)**: Trying to shove the entire mountain into your mouth at once. You will choke (Out of Memory).
- **Batch Size = 64 (Mini-Batch GD)**: Eating with a spoon. Efficient and manageable.

The `DataLoader` is your spoon.

## Part 3: Custom Dataset (The Practice)

Let's build a fake dataset of random numbers.

In [ ]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

# Create a dataset with 1000 items
dataset = RandomDataset(size=5, length=1000)

# Test the Librarian
print(f"Item 0: {dataset[0]}")
print(f"Length: {len(dataset)}")

## Part 4: The DataLoader (The Driver)

Now let's put the Driver to work.

In [ ]:
# Create a DataLoader
loader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)

# Test the Driver
for batch in loader:
    print(f"Batch Shape: {batch.shape}")
    break # Just show the first one

## Part 5: Saving and Loading (The Save Game)

Training takes hours. You don't want to lose progress if your computer crashes.

We use `torch.save` and `torch.load`.

### `state_dict()` vs Saving the Entire Model

**Option A: Save `state_dict()` (Recommended)**
```python
torch.save(model.state_dict(), 'weights.pth')
```
- Saves only the learned parameters (weights and biases)
- Portable: works even if you change the class definition slightly
- Smaller file size
- You must define the model class before loading

**Option B: Save the entire model (Not recommended)**
```python
torch.save(model, 'model.pth')
```
- Saves everything including the class definition via Python's `pickle`
- Breaks if you move files, rename classes, or change imports
- Brittle and hard to debug

**Rule of thumb**: Always use `state_dict()`. The only time you'd save the full model is for quick prototyping when you don't care about long-term reproducibility.

### Loading with `weights_only=True` (Security)

Starting in PyTorch 2.0+, you should use `weights_only=True` when loading:
```python
model.load_state_dict(torch.load('weights.pth', weights_only=True))
```
This prevents arbitrary code execution from malicious checkpoint files (pickle deserialization attacks).

In [ ]:
# Define a simple model
model = nn.Linear(5, 1)

# Save the state dictionary (The Weights)
torch.save(model.state_dict(), 'model_weights.pth')
print("Model saved to model_weights.pth")

# Load it back
new_model = nn.Linear(5, 1)
new_model.load_state_dict(torch.load('model_weights.pth'))
print("Model loaded successfully!")

# Verify they are the same
print(f"Original Weight: {model.weight}")
print(f"Loaded Weight:   {new_model.weight}")

## ML System Design: Summary Checklist

### The Interview Template (Print This!)

```
STEP 1: CLARIFY (5 min)
  □ Scale (users, items, QPS)
  □ Latency requirements
  □ Online vs Offline
  □ Cold start problem?

STEP 2: OBJECTIVE (5 min)
  □ What to predict? (click, watch time, etc)
  □ Offline metrics (AUC, NDCG)
  □ Online metrics (revenue, engagement)

STEP 3: FEATURES (10 min)
  □ User features
  □ Item features
  □ Context features
  □ Interaction features

STEP 4: MODEL (10 min)
  □ Two-stage (retrieval + ranking)
  □ Candidate generation method
  □ Ranking model architecture

STEP 5: TRAINING (10 min)
  □ Data collection strategy
  □ Loss function
  □ Negative sampling
  □ Evaluation strategy

STEP 6: SERVING (10 min)
  □ System architecture
  □ Latency optimization
  □ Caching strategy
  □ Model update frequency
```

### Red Flags to Avoid

❌ **Don't do this**:
1. Jump straight to model architecture (skip requirements)
2. Ignore latency constraints (real-time matters!)
3. Use accuracy for imbalanced data
4. Forget about cold start
5. Not mention A/B testing
6. Ignore fairness/ethics

✅ **Do this**:
1. Ask clarifying questions
2. Discuss trade-offs (accuracy vs latency)
3. Mention production challenges (drift, scaling)
4. Show ML + Systems knowledge
5. Use concrete numbers (not "a lot of users")
6. Draw diagrams!

### Key Takeaways

1. **Two-stage is standard**: Retrieval (fast, broad) → Ranking (slow, precise)
2. **Features matter more than models**: 50 good features > fancy architecture
3. **Offline ≠ Online**: AUC ↑ doesn't guarantee revenue ↑
4. **Scale is everything**: Can't run BERT on 1B items in 100ms
5. **Always A/B test**: Never deploy without validation

### Additional Resources (What FAANG Engineers Read)

**Papers**:
- YouTube: "Deep Neural Networks for YouTube Recommendations" (2016)
- Facebook: "Practical Lessons from Predicting Clicks on Ads at Facebook" (2014)
- Netflix: "Netflix Recommendations: Beyond the 5 stars" (2012)
- Google: "Wide & Deep Learning for Recommender Systems" (2016)

**Books**:
- "Designing Machine Learning Systems" by Chip Huyen
- "Machine Learning System Design Interview" by Ali Aminian & Alex Xu

**Practice Platforms**:
- InterviewQuery.com (ML system design mock interviews)
- ByteByteGo (System design fundamentals)

---

## Final Thoughts

**You now have the complete toolkit for FAANG ML interviews:**

1. ✅ **PyTorch fundamentals** (tensors, autograd, models)
2. ✅ **Deep learning architectures** (CNNs, RNNs, Transformers, ResNet, ViT)
3. ✅ **Optimization techniques** (AdamW, AMP, LR warmup, quantization)
4. ✅ **Production deployment** (TorchServe, FastAPI, Docker)
5. ✅ **Real-world challenges** (imbalanced data, focal loss, metrics)
6. ✅ **ML System Design** (recommendations, ranking, search, ads)

**The difference between you and other candidates**: You understand both the **code** AND the **systems**.

**Next steps**:
1. Practice coding 1 architecture from scratch daily
2. Mock ML system design interviews weekly
3. Read 1 FAANG ML paper per week
4. Build 1 end-to-end project (training → deployment)

**You are ready for FAANG 2025.** 🚀